In [66]:
# a one-off notebook to augment the 4 second dataset with clips where no sound occurs!

In [67]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import numpy as np
import scipy.io.wavfile
import csv 
import collections
sys.path.append(os.path.expanduser('~/projects/engaged_hackathon/'))
from engaged.features import wave
where_to_save = '/home/michael/projects/engaged_hackathon_data/detection/training_data/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
# load in the annotations
base_path = '/home/michael/projects/engaged_hackathon_data/raw_data/one_minute_files/'
dataset_csv = csv.reader(open(base_path + 'urban_sounds_labels.csv'))

annotations = []  # list of all class info

def try_number(s):
    """Converts s to float if possible, else leaves as is"""
    try:
        return float(s)
    except ValueError:
        return s

# I'm basically reinventing pandas here - very silly
for count, line in enumerate(dataset_csv):
    if count == 0:
        header = line
        continue
        
    annotation = {label:try_number(item) for label, item in zip(header, line)}    
    annotation['length'] = \
        annotation['LabelEndTime_Seconds'] - annotation['LabelStartTime_Seconds']
    annotation['Label'] = annotation['Label'].strip().lower()
    annotations.append(annotation)
    
# group annotations by filename
file_annotations = collections.defaultdict(list)
for annotation in annotations:
    file_annotations[annotation['Filename']].append(annotation)

In [69]:
from scipy.ndimage.measurements import label as scipy_label

blank_snippets = []

In [70]:
# need to load in the 4 second files just to get the correct label for each snippet
base_4s = '/home/michael/projects/engaged_hackathon_data/raw_data/cropped_audio/'
dataset_csv = csv.reader(open(base_4s + 'below12kHz_cropped_label.txt'))

dataset_4s = collections.defaultdict(list)
for line in dataset_csv:
    annot = {'filename':line[0], 'start_time': line[1], 'label': line[2]}
    base_fname = '_'.join(annot['filename'].split('_')[:-1]) + '.wav'
    dataset_4s[base_fname].append(annot)
print dataset_4s

defaultdict(<type 'list'>, {'W84LA-013548_20130622_0130.wav': [{'start_time': ' 0.01', 'label': ' Mix traffic', 'filename': 'W84LA-013548_20130622_0130_0.wav'}, {'start_time': ' 51.14', 'label': ' Mix traffic', 'filename': 'W84LA-013548_20130622_0130_1.wav'}, {'start_time': ' 50.17', 'label': ' Mix traffic', 'filename': 'W84LA-013548_20130622_0130_2.wav'}, {'start_time': ' 53.37', 'label': ' Braking', 'filename': 'W84LA-013548_20130622_0130_3.wav'}, {'start_time': ' 53.41', 'label': ' Braking', 'filename': 'W84LA-013548_20130622_0130_4.wav'}, {'start_time': ' 41.76', 'label': ' Braking', 'filename': 'W84LA-013548_20130622_0130_5.wav'}, {'start_time': ' 42.84', 'label': ' Braking', 'filename': 'W84LA-013548_20130622_0130_6.wav'}, {'start_time': ' 42.13', 'label': ' Braking', 'filename': 'W84LA-013548_20130622_0130_7.wav'}, {'start_time': ' 43.65', 'label': ' Bus emitting', 'filename': 'W84LA-013548_20130622_0130_8.wav'}, {'start_time': ' 46.19', 'label': ' Mix traffic', 'filename': 'W84

In [75]:
where_to_save = '/home/michael/projects/engaged_hackathon_data/raw_data/blank_snippets/'
blank_annotations = []

one_min_files_with_big_blanks = set()

for fname in file_annotations.keys():

    # where we'll temp store the blank snippets for this file
    blank_snippets = []
    
    # load file and convert to spectrogram
    sr, wav = scipy.io.wavfile.read(base_path + '25_Random/' + fname)  
    
    # create label vector...
    labels = 0 * wav    
    
    found_traffic = False
    
    # loop over each annotation...
    for annot in file_annotations[fname]:
        
        # fill in the label vector
        start_point = int(annot['LabelStartTime_Seconds'] * sr)
        end_point =  int(annot['LabelEndTime_Seconds'] * sr)
        labels[start_point:end_point] += 1
        
        if annot['Label'].strip().lower() == 'mix traffic':
            found_traffic = True

    labelling, n_labels = scipy_label(labels==0)
        
    for label in range(1, n_labels):

        idxs = labelling == label
        length_in_s = float(idxs.sum()) / float(sr)
        segment_start_point = np.where(idxs)[0].min()
        segment_end_point = np.where(idxs)[0].max()
        
                
        assert(labels[idxs].sum() == 0)
        
        #  check it's long enough
        if length_in_s > 4:
            
            # filling in this set - probably don't care if there is any mix traffic labelled in file...
            if found_traffic == False:
                one_min_files_with_big_blanks.add(fname)
            
            # work out how many 4 second snippets we could get out of this blank segment
            max_snippets = np.floor(length_in_s / 4.0)
            
            # now linearly space this many starting points between the start and end of the file
            start_points = np.linspace(
                segment_start_point, 
                segment_end_point, 
                max_snippets+1)[:-1]
            
            for start_point in start_points:

                # extract the snippet
                end_point = start_point + 4 * sr            
                snippet = wav[start_point:end_point]
                
                start_time = float(start_point) / float(sr)

                blank_snippets.append((start_time, snippet))

    # ok now let's save the snippets to disk

    # now augment a *new* annotations list with these new snippets - these can be then added later
    new_label = len(dataset_4s[fname])
    
    for start_time, snippet in blank_snippets:
        
        # generating the save path for the blank snippet
        split = fname.split('.')
        new_fname = split[0] + '_' + str(new_label) + '.wav'
        savepath = where_to_save + new_fname
        print savepath
        
        scipy.io.wavfile.write(savepath, sr, snippet)
        
        # adding to the annotations
        to_add = {'filename':new_fname, 'start_time': '%.2f' % start_time, 'label': 'blank'}
        blank_annotations.append(to_add)
        
        new_label += 1


/home/michael/projects/engaged_hackathon_data/raw_data/blank_snippets/HA86RB-13527_20130727_2357_2.wav
/home/michael/projects/engaged_hackathon_data/raw_data/blank_snippets/HA86RB-13527_20130727_2357_3.wav
/home/michael/projects/engaged_hackathon_data/raw_data/blank_snippets/HA86RB-13527_20130727_2357_4.wav
/home/michael/projects/engaged_hackathon_data/raw_data/blank_snippets/E47EN-013527_20131008_0430_4.wav
/home/michael/projects/engaged_hackathon_data/raw_data/blank_snippets/E47EN-013527_20131008_0430_5.wav
/home/michael/projects/engaged_hackathon_data/raw_data/blank_snippets/E47EN-013527_20131008_0430_6.wav
/home/michael/projects/engaged_hackathon_data/raw_data/blank_snippets/E47EN-013527_20131008_0430_7.wav
/home/michael/projects/engaged_hackathon_data/raw_data/blank_snippets/E47EN-013527_20131008_0430_8.wav
/home/michael/projects/engaged_hackathon_data/raw_data/blank_snippets/E47EN-013527_20131008_0430_9.wav
/home/michael/projects/engaged_hackathon_data/raw_data/blank_snippets/E47

In [83]:
# finally, save the annotations list to disk somewhere
with open(where_to_save + '../new_snippets.csv', 'w') as f:
    csv_writer = csv.writer(f, delimiter=',')
    
    for annot in blank_annotations:
        csv_writer.writerow(
            [annot['filename'], annot['start_time'], annot['label'], ''])

In [72]:
files = [xx for xx in one_min_files_with_big_blanks]
for xx in sorted(files):
    print xx

CR05EF-13527_20130917_1527.wav
CR05EF-13527_20130919_1733.wav
CR05EF-13527_20130920_2205.wav
E105JP-13548_20131009_0409.wav
E105JP-13548_20131010_0009.wav
E105JP-13548_20131010_0728.wav
HA53AA-13548_20130724_0452.wav
HA53AA-13548_20130728_1233.wav
HA53AA-13548_20130729_0452.wav
HA86RB-13527_20130730_0318.wav
HA86RB-13527_20130730_1113.wav
RM14-3YB-944_1_20130612_010200_000.wav
RM14-3YB-944_1_20130612_140700_000.wav
RM14-3YB-944_1_20130612_160500_000.wav
RM14-3YB-944_1_20130615_050700_000.wav
RM14-3YB-944_1_20130616_020000_000.wav
RM14-3YB-944_1_20130616_130700_000.wav
RM14-3YB-944_1_20130616_210700_000.wav
RM14-3YB-944_1_20130617_010400_000.wav
RM14-3YB-944_1_20130618_010600_000.wav
RM14-3YB-944_1_20130619_010000_000.wav
WC2H8LG-3527_20130712_1419.wav


In [73]:
# now save each of these snippets to a new file... with the appropriate name!
for annots in file_annotations.itervalues():
    print annots[0]
    sdsdsd

{'MeanAmp': 7.028996511, 'LabelStartTime_Seconds': 0.01, 'MaximumFreq_Hz': 6533.333333, 'MaxAmp': 12.86668531, 'Spec_NStep': 0.01, 'MinAmp': -0.308658414, 'LabelArea_Datapoints_Threshold': 1157035.0, 'LabelArea_DataPoints': 1157035.0, 'Label': 'mix traffic', 'Spec_x2': 5996.0, 'length': 59.95, 'AmpSD': 0.784654227, 'Spec_y1': 164.0, 'LabelEndTime_Seconds': 59.96, 'Spec_NWin': 0.03, 'Spec_x1': 1.0, 'Filename': 'W84LA-013548_20130622_0130.wav', 'LabelTimeStamp': '2015-03-20T14:29:27.021000', 'Spec_y2': 357.0, 'MinimumFreq_Hz': 100.0}


NameError: name 'sdsdsd' is not defined

In [ ]:
print len(blank_snippets)

In [ ]:
print len(file_annotations.keys())

In [ ]:
print sr